In [53]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Import ASCII text of the books "Alice in Wonderland" (downloaded from project Gutenberg) and convert all characters to lowercase so that the model does not treat capitalized words as new words.

In [47]:
# load text and convert to lowercase
filename = "./input/wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [48]:
# create mapping of unique charst to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c,i) for i, c in enumerate(chars))

Summary Statistics of the data set

In [49]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", str(n_chars))
print("Total distinct characters: ", str(n_vocab))

Total Characters:  144407
Total distinct characters:  45


A training pattern are the firtst 100 characters, the ground-truth "label" for this first training pattern then is the 101st character. This window of 100 characters then gets slided character by character.

For illustration purposes assume we would take a sequence of 5 characters, then the first training sample would be `chapt` -> `e` and the second training sample would be `hapte` -> `r`. 

The characters get converted to integers using the lookup dictionary created before.

In [50]:
# Prepare dataset
seq_length = 100
trainX = []
trainY = []
for i in range(0, n_chars-seq_length, 1):
    seq_in = raw_text[i : i + seq_length]   # in 1st iteration contains first 100 chars
    seq_out = raw_text[i + seq_length]   # in 1st iteration contains 101st char
    trainX.append([char_to_int[char] for char in seq_in])   # char is the character as string, char_to_int[char] gives the int value
    trainX
    trainY.append(char_to_int[seq_out])
n_patterns = len(trainX)
print("Total # of Patterns: " + str(n_patterns))

Total # of Patterns: 144307


To train the network we need to transform the data further. 
1. Reshape the training data to the form [samples, time steps, features].
2. Rescale the integers to the range 0-1 to better train the network if a sigmoid function is used (stay in approx linear part of the sigmoid). 
3. One-hot encode trainY so that each character in y is represented by a vector of 45 (number of distinct characters) values. The character "n" (inter value 32) is then represented by a vector of zeros except for one "1" in column 32.

In [58]:
# reshape X to [samples, time steps, features]
X = np.reshape(trainX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one-hot encode y
y = np_utils.to_categorical(trainY)

We define a LSTM model with one hidden layer and 256 memory units. Dropout is used with a probability of 20%. The output layer is a Dense layer using the softmax activation function. This outputs a probability prediction for each character.

In [60]:
model = Sequential()
model.add(LSTM(256, input_shape = (X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation = "softmax"))
model.compile(loss = "categorical_crossentropy", optimizer = "adam")

In [61]:
# define checkpoints
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor = "loss", verbose = 1, save_best_only = True, mode = "min") 
callbacks_list = [checkpoint]

In [62]:
model.fit(X, y, epochs = 4, batch_size = 128, callbacks = callbacks_list)

Epoch 1/4
144256/144307 [============================>.] - ETA: 0s - loss: 2.9600Epoch 00001: loss improved from inf to 2.95995, saving model to weights-improvement-01-2.9599.hdf5


ImportError: `save_model` requires h5py.